**1/6/18 update: using json_normalize to solve the problem**


JSON exercise  

Springboard Data Science Career track Chapter 5.2 JSON Based Data Exercises

** Submitted by: Pan Chen**

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.



# 1. Find the 10 countries with most projects


In [39]:
#1. Find the 10 countries with most projects
#import packages
import json
import pandas as pd
import numpy as np
pd.reset_option('all') 

#load json file
df = pd.read_json('data/world_bank_projects.json')

#find the value counts of each country
print(df['countryshortname'].value_counts(dropna=False)[0:10])

'''(['China', 'Indonesia', 'Vietnam', 'India', 'Yemen, Republic of',
       'Morocco', 'Bangladesh', 'Nepal', 'Africa', 'Mozambique']
       are 10 countries with most projects
       '''

height has been deprecated.

line_width has been deprecated, use display.width instead (currently both are
identical)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Nepal                 12
Bangladesh            12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64


/Users/cpkoywk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  


"(['China', 'Indonesia', 'Vietnam', 'India', 'Yemen, Republic of',\n       'Morocco', 'Bangladesh', 'Nepal', 'Africa', 'Mozambique']\n       are 10 countries with most projects\n       "

10 Countries with most projects are: 'China', 'Indonesia', 'Vietnam', 'India', 'Yemen, Republic of','Morocco', 'Bangladesh', 'Nepal', 'Africa', and 'Mozambique'

# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
# 3. Above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

Firstly, use json_normalize function to flatten "mjtheme_namecode" column and store it into a dtaframe


In [40]:
from pandas.io.json import json_normalize
# Load JSON: json_data
with open("data/world_bank_projects.json") as json_file:
    json_data=json.load(json_file)

themedf=json_normalize(json_data, 'mjtheme_namecode',['id'])
themedf=themedf.replace('', np.nan)
themedf

,code,name,id
0,8,Human development,P129828
1,11,NaN,P129828
2,1,Economic management,P144674
3,6,Social protection and risk management,P144674
4,5,Trade and integration,P145310
5,2,Public sector governance,P145310
6,11,Environment and natural resources management,P145310
7,6,Social protection and risk management,P145310
8,7,Social dev/gender/inclusion,P144665
9,7,Social dev/gender/inclusion,P144665


There are some missing values of theme name, so I will create a dictionary first, so the dataframe could look up the missing values in the dictionary and fill them in

In [41]:
import numpy as np

#get a dictionary of code and name
uniqueproj=themedf.replace('',np.nan).dropna().drop_duplicates()
uniqueproj=uniqueproj.set_index('code')
uniqueprojdict=uniqueproj.to_dict('dict')
uniqueprojdict=uniqueprojdict['name']
uniqueprojdict


{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

Look up the missing values in the dictionary and fill up the dataframe 'themedf':

In [42]:
themedf.name = themedf.name.fillna(themedf.code.map(uniqueprojdict))
themedf['mjtheme_namecode']=''
for ind, row in themedf.iterrows():
    themedf['mjtheme_namecode'][ind]={'code':themedf['code'][ind],'name':themedf['name'][ind]}


Reconstruct the dataframe by 'id'

In [50]:
agg=themedf.groupby('id')
aggdf=agg['mjtheme_namecode'].apply(lambda x: [x])
aggdf=aggdf.reset_index()
aggdf.mjtheme_namecode=aggdf.mjtheme_namecode.apply(lambda x: list(x[0]))

Put the new 'mjtheme_namecode' column back to the original dataframe

In [51]:
df=df.sort_values(by='id')
aggdf=aggdf.sort_values(by='id')

df['mjtheme_namecode']=aggdf['mjtheme_namecode']

The top 10 Themes:

In [49]:
themedf['name'].value_counts()

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: name, dtype: int64

The top 10 themes are: 'Environment and natural resources management', 'Rural development',
       'Human development', 'Public sector governance',
       'Social protection and risk management',
       'Financial and private sector development',
       'Social dev/gender/inclusion', 'Trade and integration',
       'Urban development', 'Economic management' and 'Rule of law'.

The new df without NA is still called 'df'

In [47]:
df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
65,{'$oid': '52b213b38594d8a2be17c7c1'},2014,August,2013-08-06T00:00:00Z,"BURUNDI,RWANDA,TANZANIA",2020-12-31T00:00:00Z,Africa!$!3A,3A,Africa,Africa,...,LH,IBRD,Active,N,"{'Percent': 15, 'Name': 'Infrastructure servic...","[{'code': '39', 'name': 'Infrastructure servic...","47,79,62,39",339900000,339900000,http://www.worldbank.org/projects/P075941/nels...
145,{'$oid': '52b213b38594d8a2be17c811'},2013,June,2013-06-07T00:00:00Z,GOVERNMENT OF CHILE,2007-12-31T00:00:00Z,Republic of Chile!$!CL,CL,Republic of Chile,Chile,...,"BL,AT,AZ",IBRD,Closed,N,"{'Percent': 26, 'Name': 'Biodiversity'}","[{'code': '80', 'name': 'Biodiversity'}, {'cod...","83,81,60,80",0,5860000,http://www.worldbank.org/projects/P085621/chil...
100,{'$oid': '52b213b38594d8a2be17c7e4'},2013,June,2013-06-27T00:00:00Z,MINISTRY OF FINANCE,2021-12-31T00:00:00Z,Republic of Kazakhstan!$!KZ,KZ,Republic of Kazakhstan,Kazakhstan,...,"BC,AZ,AB,AI",IBRD,Active,N,"{'Percent': 1, 'Name': 'Water resource managem...","[{'code': '85', 'name': 'Water resource manage...","78,79,77,85",102900000,102900000,http://www.worldbank.org/projects/P086592/seco...
287,{'$oid': '52b213b38594d8a2be17c89f'},2013,March,2013-03-14T00:00:00Z,"MALAWI, MOZAMBIQUE, ZAMBIA",2020-01-31T00:00:00Z,Africa!$!3A,3A,Africa,Africa,...,"AZ,AI,AH,BL,AB",IBRD,Active,N,"{'Percent': 80, 'Name': 'Rural services and in...","[{'code': '78', 'name': 'Rural services and in...","47,48,78",90000000,90000000,http://www.worldbank.org/projects/P094183/agri...
460,{'$oid': '52b213b38594d8a2be17c94c'},2013,September,2012-09-25T00:00:00Z,THE FEDERAL REPUBLIC OF NIGERIA,2018-12-31T00:00:00Z,Federal Republic of Nigeria!$!NG,NG,Federal Republic of Nigeria,Nigeria,...,"BL,BH,TI",IBRD,Active,N,"{'Percent': 50, 'Name': 'Trade facilitation an...","[{'code': '49', 'name': 'Trade facilitation an...","78,49",170000000,170000000,http://www.worldbank.org/projects/P095003/ng-r...
328,{'$oid': '52b213b38594d8a2be17c8c8'},2013,January,2013-01-31T00:00:00Z,MINISTRY OF FINANCE,2017-12-31T00:00:00Z,Russian Federation!$!RU,RU,Russian Federation,Russian Federation,...,"BO,FH,FI",IBRD,Closed,N,"{'Percent': 10, 'Name': 'International financi...","[{'code': '42', 'name': 'International financi...","40,42",20000000,20000000,http://www.worldbank.org/projects/P095554/micr...
43,{'$oid': '52b213b38594d8a2be17c7ab'},2014,September,2013-09-12T00:00:00Z,SAO TOME & PRINCIPE,2010-06-30T00:00:00Z,Democratic Republic of Sao Tome and Prin!$!ST,ST,Democratic Republic of Sao Tome and Prin,Sao Tome and Principe,...,LC,IBRD,Closed,N,"{'Percent': 100, 'Name': 'Other public sector ...","[{'code': '30', 'name': 'Other public sector g...",30,0,100000,http://www.worldbank.org/projects/P098847/sao-...
389,{'$oid': '52b213b38594d8a2be17c905'},2013,November,2012-11-27T00:00:00Z,PALESTINE LIBERATION ORGANIZATION (PLO),2016-06-30T00:00:00Z,West Bank and Gaza!$!GZ,GZ,West Bank and Gaza,West Bank and Gaza,...,"BW,WV,WC",IBRD,Active,N,"{'Percent': 24, 'Name': 'Infrastructure servic...","[{'code': '39', 'name': 'Infrastructure servic...","71,39",0,6400000,http://www.worldbank.org/projects/P101289/gz-g...
311,{'$oid': '52b213b38594d8a2be17c8b7'},2013,February,2013-02-21T00:00:00Z,GOVERNMENT OF PAPUA NEW GUINEA,2018-01-31T00:00:00Z,The Independent State of Papua New Guine!$!PG,PG,The Independent State of Papua New Guine,Papua New Guinea,...,"BU,LR",IBRD,Active,NaN,"{'Percent': 48, 'Name': 'Infrastructure servic...","[{'code': '39', 'name': 'Infrastructure servic...","78,39",7300000,7300000,http://www.worldbank.org/projects/P101578/png-...
452,{'$oid': '52b213b38594d8a2be17c944'},2013,September,2012-09-27T00:00:00Z,GOVERNMENT OF INDIA,2017-12-31T00:00:00Z,Republic of India!$!IN,IN,Republic of India,India,...,"JA,WZ,BH",IBRD,Active,N,"{'Percent':